In [1]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


import pandas as pd
# from datasets import Dataset
import json


import os
# import google.generativeai as genai

/home/lgrneto/agent-pca/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carregar Vector Store e configurar funções de retrieval e geração de resposta

In [2]:
#sentence_transformers
#rank-bm25
#langchain_text_splitters
#chromadb

In [3]:
from src.vectorstore.hybrid_vector_store import HybridVectorStore
from src.config.settings import settings

#### Chroma Vector Store
#### Para criar do zero, ver Vector_Store.ipynb
store = HybridVectorStore(
    persist_path=settings.DATA_DB,
    embedding_model=settings.EMBEDDING_MODEL
)

In [28]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = OllamaLLM(model="deepseek-r1:latest", temperature=0)

prompt = ChatPromptTemplate.from_template(
"""
Você é um ACP - um Agente Conversacional Pedagógico especializado em ajudar estudantes com dúvidas relacionadas a conteúdos acadêmicos.
Sua função é fornecer respostas claras e informativas com base no material de estudo fornecido pelo contexto.
Inclua todas as informações relevantes do contexto em suas respostas, evitando suposições ou informações externas.
Suas respostas devem ser sempre em português brasileiro e devem usar um tom leve.

Se não encontrar a resposta no contexto, diga:
"Nenhuma informação disponível no contexto."
            
Contexto:
{context}

Pergunta:
{question}
""")

chain = (
        prompt
        | llm
        | StrOutputParser()
    )


In [ ]:
#Função que integra busca e geração de resposta

def rag_pipeline(question: str):
    docs = store.hybrid_search(question, top_k=5)
    context = "\n\n".join([d[1] for d in docs])

    chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke({"context": context, "question": question})

response = rag_pipeline("Qual a diferença entre Data Scientist e ML Engineer?")
print(response)

Ótimo questionamento! Vamos esclarecer a diferença entre esses dois papéis importantes no ecossistema de Machine Learning.

De acordo com o contexto:

1.  **Foco:** O Data Scientist se concentra principalmente em extrair insights e desenvolver/refinar modelos de Machine Learning. O ML Engineer se concentra na parte de engenharia, responsável por construir, implementar e manter esses modelos.

2.  **Expertise:** O Data Scientist tem uma forte base em análise, matemática e estatística. O ML Engineer tem uma compreensão mais profunda de princípios de engenharia de software, infraestrutura e serviços em nuvem.

3.  **Colaboração:** Ambos precisam de habilidades de comunicação, mas o ML Engineer tende a trabalhar mais com engenheiros de software, DevOps e profissionais de TI para integrar os modelos de Machine Learning aos sistemas existentes.

Em resumo, embora ambos contribuam para o ecossistema de Machine Learning, o Data Scientist é mais focado na análise e modelagem estatística, enquan

In [30]:
# Função apenas de recuperação de documentos

def real_retrieval(question: str):
    docs = store.hybrid_search(question, top_k=5)
    return [docs[i][1] for i in range(len(docs))]

retrieve = real_retrieval("Qual a diferença entre Data Scientist e ML Engineer?")
print(retrieve)

['the machine learning ecosystem.\nWhat is a Data Scientist ?\nA Data Scientist is an expert in extracting valuable insights from large\nvolumes of data. With a strong background in mathematics, statistics, and\nprogramming, Data Scientists analyze and interpret data to solve complex\nproblems and support better decision-making within an organization. They\nwork closely with business stakeholders to understand their objectives and\ndevelop machine learning models to help achieve those goals.\nKey responsibilities of a Data Scientist include:\nGathering, cleaning, and preprocessing data\nDeveloping machine learning models and algorithms\nEvaluating model performance and optimizing as needed\nCommunicating findings and insights to business stakeholders\nWhat is an ML Engineer?\nAn ML Engineer, or Machine Learning Engineer, is a professional who\ndesigns, develops, and implements machine learning models. They work\nclosely with data scientists to translate prototypes into efficient and', 

## Usando Deep Eval para métricas das gerações de respostas contidas no ground_truth

Datasets completos e as avaliações estão dentro da pasta artifacts.

In [34]:
from dotenv import load_dotenv
load_dotenv()

from deepeval.test_case import LLMTestCase
import json
import pandas as pd

In [35]:
####### Gera métricas para cada uma das entradas da tabela de ground_truth.

test_cases = []

df = pd.read_csv("data/ground_truth/ground_truth_mlops.csv", encoding="latin-1")
for index, row in df.iterrows():
    test_cases.append(LLMTestCase(input=row["question"], expected_output=row["answer"], actual_output=rag_pipeline(row["question"]), retrieval_context=real_retrieval(row["question"])))

### Salva os test_cases gerados em um arquivo JSON. Descomentar para usar.

test_cases_dicts = [
    {
        "input": tc.input,
        "expected_output": tc.expected_output,
        "actual_output": tc.actual_output,
        "retrieval_context": tc.retrieval_context
    }
    for tc in test_cases
]

# Salva a lista de dicionários em um arquivo JSON
with open("artifacts/eval_metrics/deepseek-r1/test_cases_mapeados.json", "w", encoding="utf-8") as f:
    json.dump(test_cases_dicts, f, ensure_ascii=False, indent=2)

In [36]:
# ####### Carrega o JSON salvo anteriormente, caso necessário. Descomentar para usar

# with open("artifacts/eval_metrics/gpt-oss/test_cases_mapeados.json", "r", encoding="utf-8") as f:
#     test_cases_dicts = json.load(f)

# # Recria a lista de LLMTestCase
# test_cases = [
#     LLMTestCase(
#         input=tc["input"],
#         expected_output=tc["expected_output"],
#         actual_output=tc["actual_output"],
#         retrieval_context=tc["retrieval_context"]
#     )
#     for tc in test_cases_dicts
# ]

In [37]:
from deepeval.models import OllamaModel

# Modelo usado para geração de métricas que necessitem de LLM. 
# PS: Só consegui fazer o llama3 usar aqui, o gpt-oss e deepseek-r1 deram erro de timeout repetidamente.

model = OllamaModel(
    model="llama3:latest",
    base_url="http://localhost:11434",
    temperature=0
)

In [38]:
from deepeval.metrics import (
  ContextualRelevancyMetric,
  ContextualRecallMetric,
  ContextualPrecisionMetric,
  AnswerRelevancyMetric,
  FaithfulnessMetric
)
from deepeval import evaluate
from deepeval.evaluate import AsyncConfig


contextual_precision = ContextualPrecisionMetric(model=model)
contextual_recall = ContextualRecallMetric(model=model)
contextual_relevancy = ContextualRelevancyMetric(model=model)
answer_relevancy = AnswerRelevancyMetric(threshold=0.8, model=model)
faithfulness = FaithfulnessMetric(model=model)

async_config = AsyncConfig(
    run_async=True,
    throttle_value=5,  
    max_concurrent=1,
)

results = []

for tc in test_cases:
    tc_results = evaluate([tc], 
                          metrics=[contextual_precision, contextual_recall, contextual_relevancy, answer_relevancy, faithfulness],
                          async_config=async_config,
                          )
    results.append(tc_results.model_dump()['test_results'][0])
    
# results = evaluate([test_cases[0]], metrics=[contextual_precision, contextual_recall, contextual_relevancy, answer_relevancy, faithfulness])

results

✨ You're running DeepEval's latest Contextual Precision Metric! (using llama3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using llama3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using llama3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using llama3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using llama3:latest (Ollama), strict=False, 
async_mode=True)...

...

✨ You're running DeepEval's latest Faithfulness Metric! (using llama3:latest (Ollama), strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Contextual Precision (score: 0.5, threshold: 0.5, strict: False, evaluation model: llama3:latest (Ollama), reason: The score is 0.50 because irrelevant nodes (nodes 1 and 3) are ranked lower than relevant nodes (nodes 2 and 4), indicating that the model correctly prioritizes contexts with information about 'champion' and 'challenger' models., error: None)
  - ✅ Contextual Recall (score: 0.8888888888888888, threshold: 0.5, strict: False, evaluation model: llama3:latest (Ollama), reason: The score is 0.89 because the contextual recall accurately captures the relationships between sentences in the expected output, with most sentences being attributed to nodes in the retrieval context, except for one standalone statement that doesn't require specific context., error: None)
  - ✅ Contextual Relevancy (score: 0.7567567567567568, threshold: 0.5, strict: False, evaluation model: llama3:latest (Ollama), reason: The score is 0.76 because the retrieval context contains re

⚠ WARNING: No hyperparameters logged.
» ]8;id=766391;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 53.97s | token cost: 0.0 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

[{'name': 'test_case_0',
  'success': True,
  'metrics_data': [{'name': 'Contextual Precision',
    'threshold': 0.5,
    'success': True,
    'score': 0.6396524110809825,
    'reason': "The score is 0.64 because irrelevant nodes (nodes 2, 4, 6, and 8) are correctly ranked lower than relevant nodes, as they lack direct connections to the expected output of defining MLOps. The first node's relevance is evident from its mention of 'MLOps', while subsequent 'yes' verdicts provide more specific explanations aligning with the expected output.",
    'strict_mode': False,
    'evaluation_model': 'llama3:latest (Ollama)',
    'error': None,
    'evaluation_cost': 0.0,
    'verbose_logs': 'Verdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": "The context mentions \'MLOps\' which is directly related to the expected output."\n    },\n    {\n        "verdict": "no",\n        "reason": "The text does not provide any information about what MLOps is, it only provides a definition and exp

In [39]:
# Salvar em JSON resultados da avaliação acima

with open("artifacts/eval_metrics/deepseek-r1/results_deepeval.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

In [41]:
# # Carrega o JSON salvo anteriormente
# with open("results_deepeval.json", "r", encoding="utf-8") as f:
#     results = json.load(f)